In [1]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationBufferWindowMemory(k=5, return_messages=True, memory_key="history")

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    streaming=True,
)

examples = [
    {"movie": "The Avengers", "answer": "🦸‍♂️🛡️🌍"},
    {"movie": "Interstellar", "answer": "🚀🌌⏳"},
    {"movie": "The Lord of the Rings", "answer": "🧙‍♂️💍🌋"},
]

example_prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "human",
            "Describe the film ‘{movie}’ in 3 emojis.",
        ),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        few_shot_prompt,
        MessagesPlaceholder(variable_name="history"),
        (
            "human",
            "Describe the film ‘{movie}’ in 3 emojis.",
        ),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(movie: str):
    result = chain.invoke({"movie": movie})
    print(result)
    memory.save_context(
        {"input": movie},
        {"output": result.content},  # type: ignore
    )



In [2]:
invoke_chain("The Dark Knight")

content='🦇🃏💥'


In [3]:
invoke_chain("Inception")

content='🧠🌀🌌'


In [8]:
ask_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        ("human", "List the film titles I've asked for so far"),
    ]
)

ask_chain = RunnablePassthrough.assign(history=load_memory) | ask_prompt | chat

ask_chain.invoke({}).content

'1. The Dark Knight\n2. Inception'